In [62]:
import json
import folium
from folium.plugins import HeatMap
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import io
import os
import imageio
from nextbike_functions import *

In [63]:
def two_hours_forward(timestamp_string: str):
    date, time = timestamp_string.split(' ')
    year, month, day = date.split('-')
    hour, minute, second = time.split(':')
    date = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))
    date += datetime.timedelta(hours=2)
    return date

For some reason, function defined above doesn't work, when declared in and imported from external file

In [64]:
data = []
timestamp_dictionary = dict()
filenames = []

i = 0
# Extracting stations info for stations in Warsaw
with open('json_files/places_and_bikes.json', 'r', encoding='utf8') as file:
    for line in file:
        data_piece = json.loads(line)
        if data_piece['city_uid'] == 812:
            timestamp = str(two_hours_forward(data_piece['timestamp']['$date'].replace('T', ' ')[:-1])).replace(':', '.')
            if not timestamp in timestamp_dictionary.keys():
                timestamp_dictionary[timestamp] = []
            timestamp_dictionary[timestamp].append(data_piece)


In [65]:
freestanding_coordinates_data = {}
for timestamp, data in timestamp_dictionary.items():
    freestanding_coordinates_data[timestamp] = []
    for station in data:
        if station['bike']:
            freestanding_coordinates_data[timestamp].append((station['lat'], station['lng']))

In [67]:
used_font = ImageFont.truetype('arial.ttf', 30)
target_folder = 'heatmaps/'

if not os.path.exists(target_folder):
    os.mkdir(target_folder)

for timestamp, freestanding_coordinates in freestanding_coordinates_data.items():
    m = folium.Map(location=[52.2265, 21.0196], zoom_start=14)
    HeatMap(freestanding_coordinates).add_to(m)
    filename = target_folder + timestamp + '.png'
    img_data = m._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    
    I1 = ImageDraw.Draw(img)
    I1.text((img.size[0]-350, img.size[1]-80), timestamp, font = used_font, fill = (0, 0, 0))
    
    img.save(filename)
    filenames.append(filename)
    print(os.path.basename(filename), "done")

images = []
for filename in filenames:
    images.append(imageio.v2.imread(filename))
imageio.mimsave('movie.gif', images)

2024-06-28 15.05.00.png done
2024-06-28 15.10.00.png done
2024-06-28 15.15.00.png done
2024-06-28 15.20.00.png done
2024-06-28 15.25.00.png done
2024-06-28 15.30.00.png done
2024-06-28 15.35.00.png done
